In [1]:
# Importing relevant libs
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
# This function receives a url and return a list of headlines. Please notice the function is independent of specific
# website, therefore can be used to extract headlines for other sites besides 'haaretz' and 'israel hayom' if needed.
def extract_news_headlines_from_url(url):
    
    headers = {'User-agent': 'Mozilla/5.0'}
    request = requests.get(url, headers=headers)
    html = request.content
    soup = BeautifulSoup(html, 'html.parser')
    
    # Get all the headers
    headers = soup.find_all('h3')
    
    # Extract headline text from each header
    headlinesList = list(map(lambda header: header.text.strip().lower(), headers))
    return headlinesList

In [3]:
# Saving urls for both haaretz and israel hayom:
haaretz_url = 'https://www.haaretz.com'
israelHayom_url = 'https://www.israelhayom.com'

# Extracting news headlines for each site
haaretz_headlines = extract_news_headlines_from_url(haaretz_url)
israelHayom_headlines = extract_news_headlines_from_url(israelHayom_url)

In [4]:
# Function for saving relevant occurences of key words for each headline
def headlineInformation(headline, target):
    headlineInfo = []
    headlineInfo.append(headline)
    headlineInfo.append(len(headline))
    headlineInfo.append(headline.count(':'))
    headlineInfo.append(headline.count('-'))
    headlineInfo.append(headline.count('netanyahu'))
    headlineInfo.append(headline.count('biden'))
    headlineInfo.append(headline.count('trump'))
    headlineInfo.append(headline.count('putin'))
    headlineInfo.append(headline.count('kill'))
    headlineInfo.append(headline.count('israel'))
    headlineInfo.append(headline.count('palestin'))
    headlineInfo.append(headline.count('china'))
    headlineInfo.append(headline.count('russia'))
    headlineInfo.append(headline.count('iran'))
    headlineInfo.append(headline.count('climate'))
    headlineInfo.append(target)
    
    return headlineInfo

In [5]:
# Declaring a list for saving headlines info and target (israelHayom/Haaretz)
headlines_with_info_and_target = []

# Generating and saving info for haaretz headlines
for headline in haaretz_headlines:
    headlines_with_info_and_target.append(headlineInformation(headline, 0))

# Generating and saving info for israel hayom headlines
for headline in israelHayom_headlines:
    headlines_with_info_and_target.append(headlineInformation(headline, 1))

# Creating a dataframe from the generated information, with target values
headlines_df = pd.DataFrame(headlines_with_info_and_target, columns=['Headlines', 'headline length',
                                                  'Occurences of ":"', 'Occurences of "-"',
                                                  'Occurences of "netanyahu"' ,'Occurences of "biden"',
                                                  'Occurences of "trump"', 'Occurences of "putin"',
                                                  'Occurences of "kill"', 'Occurences of "israel"',
                                                  'Occurences of "palestin"', 'Occurences of "china"',
                                                  'Occurences of "russia"', 'Occurences of "iran"',
                                                  'Occurences of "climate"', 'Target: haaretz=0/israelHayom=1'])

print (headlines_df)

                                             Headlines  headline length  \
0    what do putin, mbs and netanyahu have in commo...               80   
1    israel's new gov't will have to back up bellig...               87   
2    netanyahu is just a different shade of the sam...               64   
3    the kibbutz movement to the right: why even ki...               75   
4    iran arrests person they say linked to israel'...               54   
..                                                 ...              ...   
235  israeli high-tech companies resume recruitment...               57   
236                                        archaeology               11   
237                                         newsletter               10   
238                                               menu                4   
239                                           about us                8   

     Occurences of ":"  Occurences of "-"  Occurences of "netanyahu"  \
0                    0     

In [6]:
# Splitting into feature values and target values and dropping the string headlines for decision tree model
target = headlines_df['Target: haaretz=0/israelHayom=1']
features_matrix = headlines_df.drop(['Headlines','Target: haaretz=0/israelHayom=1'],axis=1)

# Splitting into train and test- with random in order to get different values each time
features_matrix_train, features_matrix_test, target_values_train, target_values_test = train_test_split(
                                                                                features_matrix, target,
                                                                                test_size=0.2, random_state=42)

In [7]:
# Define a function that will create a decision tree that classifies the headline (haaretz=0/israelHayom=1) 
def get_decision_tree_classifier(features_matrix_train, target_values_train):
    classifier = DecisionTreeClassifier(criterion="gini", min_impurity_decrease = 0.005)
    classifier.fit(features_matrix_train, target_values_train)
    return classifier

# Call the function
decision_tree_classifier = get_decision_tree_classifier(features_matrix_train, target_values_train)

In [8]:
# Define a function that takes the trained model and test it on unseen data (i.e., test data)
def get_decision_tree_predictions(classifier, features_matrix_test):
    target_predictions = classifier.predict(features_matrix_test)
    return target_predictions

# Call the function
target_values_decision_tree_predictions = get_decision_tree_predictions(decision_tree_classifier, features_matrix_test)

In [9]:
# Define a function that measure the accuracy
def measure_accuracy(test, pred):
    accuracy = metrics.accuracy_score(test,pred)
    return accuracy

# Call the function
decision_tree_classifier_accuracy = measure_accuracy(target_values_test, target_values_decision_tree_predictions)

# Show the Accuracy value
print ('Decision tree model accuracy is:', decision_tree_classifier_accuracy * 100,'%')

Decision tree model accuracy is: 72.91666666666666 %


In [10]:
plt.figure(figsize=(250,200))
fig = tree.plot_tree(decision_tree_classifier, feature_names=features_matrix.columns,class_names=['haaretz', 'israelHayom'],filled=True)